# Fusion AVANCÉE : TDMS avec Resets (Essais Invalidés)

**Version Spéciale v2** : Ignore le timestamp TDMS si un reset est détecté.

**Logique** :
1. **Detection Reset** : Si trouvé, on considère que le fichier contient une partie invalide + concaténation.
2. **Ignore TDMS Start** : Le timestamp du fichier metadata est inutile (car temps d'arrêt inconnu).
3. **Forced Sync** : On cale le début de la partie valide sur `Xsens Start + 0.2679s`.

In [ ]:
import pandas as pd
import numpy as np
from nptdms import TdmsFile
from scipy.interpolate import interp1d
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

# --- CONFIGURATION ---
f_xsens = r'Moto_Chicane_mouille_80_P1.txt'
f_tdms = r'Moto_Chicane_mouille_80.tdms'
tdms_group = 'P1'
OUTPUT_CSV = 'Merged_Corrupted_Fixed.csv'

TDMS_FREQ = 400.0
MAGIC_OFFSET = 0.2679 # Seconde

In [ ]:
def load_xsens(path):
    print(f"--- Chargement Xsens : {path} ---")
    h_idx = None
    try:
        with open(path, 'r', errors='ignore') as f:
            for i, l in enumerate(f):
                if l.strip().startswith('PacketCounter'): h_idx=i; break
        if h_idx is None:
             with open(path, 'r', errors='ignore') as f:
                for i, l in enumerate(f): 
                    if 'UTC_Year' in l: h_idx=i; break
        if h_idx is None: raise ValueError("Header introuvable")
        
        try: df = pd.read_csv(path, sep='\t', header=h_idx)
        except: df = pd.read_csv(path, sep=r'\s+', header=h_idx)
        
        df.columns = df.columns.str.strip()
        
        # FIX Ghost Packets
        check_cols = [c for c in ['Acc_X', 'FreeAcc_E', 'Gyr_X'] if c in df.columns]
        if check_cols:
            df.dropna(subset=check_cols, how='all', inplace=True)
        
        req = ['UTC_Year', 'UTC_Month', 'UTC_Day', 'UTC_Hour', 'UTC_Minute', 'UTC_Second', 'UTC_Nano']
        if set(req).issubset(df.columns):
            df.dropna(subset=req, inplace=True)
            ts = pd.to_datetime(df[req[:-1]].astype(int).rename(columns={'UTC_Year':'year','UTC_Month':'month','UTC_Day':'day','UTC_Hour':'hour','UTC_Minute':'minute','UTC_Second':'second'}))
            df['TS_UTC'] = ts + pd.to_timedelta(df['UTC_Nano'], unit='ns')
            df.drop(columns=req, inplace=True)
            df.sort_values('TS_UTC', inplace=True)
            df.drop_duplicates(subset=['TS_UTC'], inplace=True)
            
            print(f"Xsens chargé : {len(df)} lignes.")
            return df
        else: raise ValueError("Colonnes UTC manquantes")
    except Exception as e: print(f"Erreur Xsens: {e}"); return pd.DataFrame()

In [ ]:
def load_tdms_special(path, group_name, xsens_start_ref=None):
    print(f"\n--- Chargement TDMS SPECIAL '{group_name}' ---")
    try:
        tdms = TdmsFile.read(path)
        target_group = None
        for g in tdms.groups():
            if g.name == group_name: target_group = g; break
        if not target_group: print("Groupe non trouvé"); return pd.DataFrame()
        
        # 1. Données Brutes
        data = {}
        for c in target_group.channels(): data[c.name] = c[:]
        if not data: return pd.DataFrame()
        l_min = min(len(v) for v in data.values())
        data = {k: v[:l_min] for k,v in data.items()}
        df = pd.DataFrame(data)
        
        base_start_time = None
        
        # 2. Detection Reset
        start_index = 0
        reset_detected = False
        
        if 'Edges_RoueAR' in df.columns:
            diffs = np.diff(df['Edges_RoueAR'].values)
            resets = np.where(diffs < -100)[0]
            if len(resets) > 0:
                start_index = resets[0] + 1
                reset_detected = True
                print(f"   [SPECIAL] RESET détecté index {start_index}")
                print("   -> Mode 'Essai Invalidé' activé.")
                print("   -> IGNORAGE Metadata TDMS (concaténation inconnue).")
            else:
                print("   [Info] Pas de reset détecté. Mode classique.")

        # 3. Stratégie Timestamp
        if reset_detected and xsens_start_ref:
            # STRATEGIE FORCÉE : On se cale sur Xsens + Offset fix
            base_start_time = xsens_start_ref + timedelta(seconds=MAGIC_OFFSET)
            print(f"   [SYNC FORCÉE] TDMS Start = Xsens Start ({xsens_start_ref}) + {MAGIC_OFFSET}s")
            print(f"                 = {base_start_time}")
            
        else:
            # Mode Classique (Metadata)
            if 'Edges_RoueAR' in target_group:
                chan = target_group['Edges_RoueAR']
                if 'wf_start_time' in chan.properties:
                     base_start_time = pd.to_datetime(chan.properties['wf_start_time']).tz_localize(None)
            if not base_start_time and 'wf_start_time' in tdms.properties:
                 base_start_time = pd.to_datetime(tdms.properties['wf_start_time']).tz_localize(None)
            print(f"   [Info] Start Time Meta utilisé : {base_start_time}")
        
        # 4. Slice Data (Uniquement si Reset)
        if start_index > 0:
            df = df.iloc[start_index:].copy().reset_index(drop=True)
            
        # 5. Index Temporel 400Hz
        if base_start_time:
            time_index = pd.date_range(start=base_start_time, periods=len(df), freq=f'{1000/TDMS_FREQ}ms')
            df['TDMS_Timestamp'] = time_index
            df.set_index('TDMS_Timestamp', inplace=True)
            df.columns = [f"TDMS_{c}" for c in df.columns]
            print(f"TDMS chargé : {len(df)} lignes valides.")
            return df
        else:
            print("ERREUR: Pas de Start Time défini !")
            return pd.DataFrame()

    except Exception as e: print(f"Err {e}"); return pd.DataFrame()

In [ ]:
# FUSION
df_xsens = load_xsens(f_xsens)

if not df_xsens.empty:
    xsens_start = df_xsens['TS_UTC'].min()
    # On passe le start time xsens pour qu'il serve de référence
    df_tdms = load_tdms_special(f_tdms, tdms_group, xsens_start_ref=xsens_start)

    if not df_tdms.empty:
        print("\nFusion...")
        t_start = max(df_xsens['TS_UTC'].min(), df_tdms.index.min())
        t_end = min(df_xsens['TS_UTC'].max(), df_tdms.index.max())
        print(f"Intervalle Commun : {t_start} -> {t_end}")
        
        df_merged = df_xsens[(df_xsens['TS_UTC'] >= t_start) & (df_xsens['TS_UTC'] <= t_end)].copy()
        df_merged.set_index('TS_UTC', inplace=True)
        
        # Interpolation
        t_slave = (df_tdms.index - pd.Timestamp("1970-01-01")) // pd.Timedelta('1ns') / 1e9
        t_master = (df_merged.index - pd.Timestamp("1970-01-01")) // pd.Timedelta('1ns') / 1e9
        
        for col in df_tdms.columns:
            f = interp1d(t_slave, df_tdms[col].values, kind='linear', fill_value="extrapolate")
            df_merged[col] = f(t_master)
            
        df_merged.to_csv(OUTPUT_CSV, date_format='%d/%m/%Y %H:%M:%S.%f')
        print(f"Terminé : {OUTPUT_CSV}")
        print(df_merged.head())
else:
    print("Erreur chargement Xsens")